# Sentiment


Importazioni:

In [1]:
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from feel_it import SentimentClassifier
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report

## 1) Italian BERT Sentiment model
https://huggingface.co/neuraly/bert-base-italian-cased-sentiment 

In [2]:
# estraggo il DataFrame dal file excel
df = pd.read_excel('../POI con recensioni/POI.xlsx')

In [3]:
df

,id,POI,rating,recensione
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume..."
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...
...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina..."
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...


In [4]:
# carico il tokenizzatore
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")

In [129]:
bert_model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
bert_model.eval()  # Imposta il modello in modalità valutazione

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

1. Estrae la recensione dalla colonna recensione.
2. Codifica la recensione utilizzando un tokenizer, parte di un modello di linguaggio preaddestrato.
3. Trasforma la sequenza codificata in un tensore e lo prepara per l'input al modello.
4. Passa il tensore al modello per ottenere i logit, che sono le uscite del modello prima di applicare la funzione di attivazione.
5. Applica una funzione di softmax sui logit per ottenere le probabilità associate a ciascuna classe (probabilità negative, neutre e positive).
6. Memorizza le probabilità negative, neutre e positive ottenute nella tabella DataFrame df.
7. Determina la classe di sentimento prevista utilizzando la probabilità massima tra negative, neutre e positive.
8. Memorizza la classe di sentimento prevista nella tabella DataFrame df.

In [6]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Itera attraverso ogni riga del DataFrame 'df', mostrando una barra di progresso con tqdm
    sentence = row.recensione

    # Tokenizza la frase della recensione usando il tokenizer specificato
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

    # Crea un tensore, usa .cuda() per trasferire il tensore alla GPU se disponibile
    tensor = torch.tensor(input_ids).long().unsqueeze(0)
    if torch.cuda.is_available():
        tensor = tensor.cuda()
        bert_model.cuda()

    # Chiama il modello BERT e ottiene i logits (uscite grezze prima dell'applicazione della funzione softmax)
    with torch.no_grad():
        logits = bert_model(tensor)
    
    # Rimuove la dimensione fittizia del batch dai logits
    logits = logits.logits.squeeze(0)

    # Applica la funzione softmax per ottenere le probabilità
    proba = nn.functional.softmax(logits, dim=0)

    # Estrae le probabilità per le classi negativo, neutro e positivo
    negative, neutral, positive = proba

    # Assegna le probabilità alle colonne 'negative', 'neutral' e 'positive' nel DataFrame
    df.loc[index, 'negative'] = proba[0].item()
    df.loc[index, 'neutral'] = proba[1].item()
    df.loc[index, 'positive'] = proba[2].item()


  0%|          | 0/5335 [00:00<?, ?it/s]

100%|██████████| 5335/5335 [04:38<00:00, 19.18it/s]


In [7]:
proba

tensor([0.0504, 0.0137, 0.9359], device='cuda:0')

Il "tokenizer" è una componente fondamentale quando si lavora con modelli di linguaggio naturale (NLP). La sua funzione principale è quella di convertire il testo in una forma che il modello possa comprendere. Questo processo coinvolge solitamente la suddivisione del testo in parole o sotto-unità semantiche (come i token) e la successiva conversione di questi token in numeri, che possono essere utilizzati come input per il modello. Questo processo di tokenizzazione può anche includere altre operazioni, come la rimozione della punteggiatura, la gestione delle maiuscole e minuscole, e altro ancora.

Il "tensore", d'altra parte, è una struttura dati fondamentale nelle librerie di calcolo numerico, come TensorFlow o PyTorch, utilizzate per l'implementazione di modelli di machine learning e deep learning. In termini semplici, un tensore può essere considerato come un array multidimensionale, che può essere un vettore (1D), una matrice (2D) o una struttura multidimensionale di ordine superiore. I tensori sono essenziali poiché consentono di rappresentare i dati e le operazioni matematiche eseguite sui dati in modo efficiente e parallelo, specialmente quando si lavora con grandi quantità di dati.

In questo contesto il "tokenizer" viene utilizzato per convertire il testo delle recensioni in sequenze di token numerici, mentre il "tensore" viene utilizzato per rappresentare queste sequenze di token in una forma che il modello di machine learning possa elaborare.

In [8]:
df.head()

,id,POI,rating,recensione,negative,neutral,positive
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202


In [9]:
# Determino l'etichetta del sentiment sulla base della probabilità più alta
df['sentiment_pred'] = df[['negative', 'positive', 'neutral']].idxmax(axis=1)


In [10]:
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive
...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000208,0.001506,0.998286,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000134,0.001653,0.998213,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000364,0.003089,0.996547,positive


In [11]:
df.to_excel('../sentiment/BERT.xlsx', index=False)

In [12]:
df = pd.read_excel('../POI con recensioni/POI.xlsx')
df.rating.value_counts()

rating
5    2637
4    1551
3     710
1     249
2     188
Name: count, dtype: int64

In [13]:
df = pd.read_excel('../sentiment/BERT.xlsx')
df.sentiment_pred.value_counts()

sentiment_pred
positive    4228
neutral      584
negative     523
Name: count, dtype: int64

In [14]:
df.to_csv("../sentiment/BERT_value_counts.csv", index=False, sep='\t')

### Calcolo dell'Accuracy

In [15]:
df = pd.read_excel('../sentiment/BERT.xlsx')

In [16]:
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive
...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000208,0.001506,0.998286,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000134,0.001653,0.998213,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000364,0.003089,0.996547,positive


In [17]:
df['GS']=0
df.loc[df['rating'] >= 4, 'GS']  = 'positive'
df.loc[df['rating'] == 3, 'GS']  = 'neutral'
df.loc[df['rating'] <= 2, 'GS']  = 'negative'
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred,GS
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive,positive
...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000208,0.001506,0.998286,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000134,0.001653,0.998213,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000364,0.003089,0.996547,positive,positive


In [18]:
print(classification_report(df['GS'], df['sentiment_pred']))

              precision    recall  f1-score   support

    negative       0.62      0.74      0.67       437
     neutral       0.32      0.26      0.29       710
    positive       0.88      0.89      0.89      4188

    accuracy                           0.80      5335
   macro avg       0.61      0.63      0.62      5335
weighted avg       0.79      0.80      0.79      5335



In [19]:
# Calcola il classification report
report = classification_report(df['GS'], df['sentiment_pred'], output_dict=True)

# Creare un DataFrame pandas dal classification report
df_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/BERT_classification_report.xlsx'
df_report.to_excel(excel_filename, index=True)

## 2) FEEL - IT
https://huggingface.co/MilaNLProc/feel-it-italian-sentiment?text=Mi+piaci.+Ti+amo

Il modello Feel-it-Italian-Sentiment esegue l'analisi del sentiment sull'italiano. Viene utilizzato il modello UmBERTo sul nuovo set di dati (cioè FEEL-IT) ottenendo prestazioni all'avanguardia su diversi corpus di riferimento.

I dati sono stati raccolti annotando tweet di un’ampia gamma di argomenti. In totale, ci sono 2037 tweet annotati con un'etichetta emozionale.

PAPER: https://aclanthology.org/2021.wassa-1.8.pdf

In [103]:
sentiment_classifier = SentimentClassifier()

In [21]:
db = df[['id', 'POI', 'rating', 'recensione']]

In [22]:
db

,id,POI,rating,recensione
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume..."
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...
...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina..."
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...


In [24]:
feel_it_scores = []
for sentence in db['recensione']:
    feel_it_scores.append(sentiment_classifier.predict([sentence])[0])
db['FEEL_IT'] = feel_it_scores
df['FEEL_IT'] = feel_it_scores

C:\Users\franc\AppData\Local\Temp\ipykernel_6448\3648803721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['FEEL_IT'] = feel_it_scores


In [102]:
db = pd.read_excel('../sentiment/FEEL_IT.xlsx')

In [25]:
# Salvo i risultati in un file CSV
db.to_csv("../sentiment/FEEL_IT_results.csv")

In [53]:
# Salvo i risultati in un file Excel
db.to_excel("../sentiment/FEEL_IT.xlsx", index=False)

In [26]:
# elimino dal df principale le colonne inutili
df.drop(['negative', 'neutral', 'positive'], axis=1, inplace=True)

In [27]:
# scampio l'ordine delle colonne
df = df[['id', 'POI', 'rating', 'recensione', 'GS', 'sentiment_pred', 'FEEL_IT']]

In [28]:
#rinomino la colonna 'sentiment_pred' in 'BERT'
df.rename(columns={'sentiment_pred': 'BERT'}, inplace=True)

In [29]:
# Salvo il DataFrame in un file Excel
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)

### Calcolo dell'Accuracy

In [30]:
db['GS']=0
db.loc[df['rating'] >= 3, 'GS']  = 'positive'
db.loc[df['rating'] <= 2, 'GS']  = 'negative'
db

C:\Users\franc\AppData\Local\Temp\ipykernel_6448\4092698158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['GS']=0


,id,POI,rating,recensione,FEEL_IT,GS
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive
...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive


In [31]:
print(classification_report(db['GS'], db['FEEL_IT']))

              precision    recall  f1-score   support

    negative       0.42      0.90      0.58       437
    positive       0.99      0.89      0.94      4898

    accuracy                           0.89      5335
   macro avg       0.71      0.90      0.76      5335
weighted avg       0.94      0.89      0.91      5335



In [32]:
# Calcola il classification report
report = classification_report(db['GS'], db['FEEL_IT'], output_dict=True)

# Creare un DataFrame pandas dal classification report
db_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/FEEL_IT_classification_report.xlsx'
db_report.to_excel(excel_filename, index=True)

## 3) MULTILINGUAL: bert-base-multilingual-uncased-sentiment
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment 

Si tratta di un modello bert-base-multilingue-uncased ottimizzato per l'analisi del sentiment sulle recensioni dei prodotti in sei lingue: inglese, olandese, tedesco, francese, spagnolo e **italiano**. Prevede il sentiment della recensione sotto forma di numero di stelle (tra 1 e 5).

In [117]:
# Load model directly from the Hugging Face Hub

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
multilingual_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment").to("cuda")

In [34]:
# Crea colonne per memorizzare le probabilità
df['1'] = 0
df['2'] = 0
df['3'] = 0
df['4'] = 0
df['5'] = 0

    # Itera attraverso ogni riga del DataFrame
for index, row in tqdm(df.iterrows()):
    sentence = row.recensione
        
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)
        
    # Create tensor, use .cuda() to transfer the tensor to GPU
    tensor = torch.tensor(input_ids).long()
    # Fake batch dimension
    tensor = tensor.unsqueeze(0).to("cuda")

    # Call the model and get the logits
    logits = multilingual_model(tensor)
    # Remove the fake batch dimension
    logits = logits.logits.squeeze(0)

    # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
    proba = nn.functional.softmax(logits, dim=0)

    # Aggiungi le probabilità alle colonne appropriate
    proba = proba.to("cpu")
    df.loc[index, '1'] = proba[0].item()
    df.loc[index, '2'] = proba[1].item()
    df.loc[index, '3'] = proba[2].item()
    df.loc[index, '4'] = proba[3].item()
    df.loc[index, '5'] = proba[4].item()

# Calcola il sentimento prevalente
df['MULTILINGUAL'] = df[['1', '2', '3', '4', '5']].idxmax(axis=1)
db = df[['id', 'POI', 'rating', 'recensione', 'GS', '1', '2', '3', '4', '5', 'MULTILINGUAL']]
df.drop(columns=['1', '2', '3', '4', '5'], inplace=True)

# Tentare di convertire i valori della colonna 'MULTILINGUAL' in numerici, ignorando gli errori
df['sentiment_pred_numeric'] = pd.to_numeric(df['MULTILINGUAL'], errors='coerce')

# Applicare le trasformazioni solo ai valori numerici
df.loc[df['sentiment_pred_numeric'] >= 4, 'MULTILINGUAL'] = 'positive'
df.loc[df['sentiment_pred_numeric'] <= 2, 'MULTILINGUAL'] = 'negative'
df.loc[df['sentiment_pred_numeric'] == 3, 'MULTILINGUAL'] = 'neutral'

# Rimuovere la colonna temporanea utilizzata per la conversione
df.drop(columns=['sentiment_pred_numeric'], inplace=True)

0it [00:00, ?it/s]

5335it [05:08, 17.29it/s]


In [35]:
db

,id,POI,rating,recensione,GS,1,2,3,4,5,MULTILINGUAL
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.004849,0.009446,0.063394,0.392164,0.530147,5
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.001022,0.001809,0.033151,0.438019,0.525999,5
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.004407,0.063242,0.514002,0.385526,0.032823,3
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.010631,0.089289,0.340011,0.504914,0.055155,4
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.002611,0.004687,0.043096,0.394785,0.554821,5
...,...,...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.001865,0.002672,0.008780,0.132900,0.853783,5
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.127622,0.136910,0.161904,0.224093,0.349471,5
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.003115,0.006855,0.028588,0.260003,0.701439,5
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.014572,0.037150,0.088022,0.334005,0.526252,5


In [36]:
# Salvo i risultati in un file CSV
db.to_csv("../sentiment/MULTILINGUAL_results.csv")

In [37]:
# Salvo il DataFrame in un file Excel
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)

### Calcolo dell'Accuracy

In [71]:
db = pd.read_csv('../sentiment/MULTILINGUAL_results.csv')

In [72]:
db['MULTILINGUAL'][0]

5

In [73]:
# converto i valori di 'MULTILINGUAL' in numerici
db['MULTILINGUAL'] = db['MULTILINGUAL'].map({5: 'positive', 4: 'positive', 3: 'neutral', 1: 'negative', 2: 'negative'})

In [74]:
db

,Unnamed: 0,id,POI,rating,recensione,GS,1,2,3,4,5,MULTILINGUAL
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.004849,0.009446,0.063394,0.392164,0.530147,positive
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.001022,0.001809,0.033151,0.438019,0.525999,positive
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.004407,0.063242,0.514002,0.385526,0.032823,neutral
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.010631,0.089289,0.340011,0.504914,0.055155,positive
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.002611,0.004687,0.043096,0.394785,0.554821,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.001865,0.002672,0.008780,0.132900,0.853783,positive
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.127622,0.136910,0.161904,0.224093,0.349471,positive
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.003115,0.006855,0.028588,0.260003,0.701439,positive
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.014572,0.037150,0.088022,0.334005,0.526252,positive


In [78]:
db.to_csv("../sentiment/MULTILINGUAL_results.csv", index=False)

In [75]:
print(classification_report(db['GS'], db['MULTILINGUAL']))

              precision    recall  f1-score   support

    negative       0.63      0.75      0.69       437
     neutral       0.39      0.28      0.33       710
    positive       0.89      0.92      0.91      4188

    accuracy                           0.82      5335
   macro avg       0.64      0.65      0.64      5335
weighted avg       0.81      0.82      0.81      5335



In [76]:
# Creare un DataFrame pandas dal classification report
db_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/MULTILINGUAL_classification_report.xlsx'
db_report.to_excel(excel_filename, index=True)

## BMA

Bayesian Model Averaging (BMA):

**Descrizione**: Una tecnica statistica utilizzata per migliorare le previsioni o le inferenze combinando diversi modelli. Anziché scegliere un singolo modello, BMA prende in considerazione molti modelli possibili e pondera le loro previsioni in base alla probabilità che ciascun modello sia corretto.

**Applicazioni**: Utilizzata in varie discipline tra cui economia, epidemiologia, e machine learning per ottenere previsioni più robuste.

**Principi di BMA**: 
- **Incertezza del Modello**: BMA riconosce che non c'è un singolo modello che sia perfetto per rappresentare i dati. Invece di selezionare un solo modello, considera una gamma di modelli possibili.
- **Pesi Bayesiani**: Ogni modello è ponderato in base alla sua probabilità a posteriori, che riflette quanto bene il modello si adatta ai dati e quanto è plausibile a priori.
- **Combina Previsioni**: Le previsioni finali sono ottenute combinando le previsioni di tutti i modelli ponderate per la loro probabilità a posteriori.

**Vantaggi di BMA:**
- **Robustezza**: Migliora la robustezza delle previsioni riducendo il rischio di fare affidamento su un singolo modello che potrebbe essere mal specificato.
- **Incorpora Incertezza**: Fornisce una misura più realistica dell'incertezza nelle previsioni incorporando la variabilità tra modelli.

In [2]:
from bma_ import bma_test

In [3]:
bert = pd.read_excel('../sentiment/BERT.xlsx')
multilingual = pd.read_csv('../sentiment/MULTILINGUAL_results.csv')
df = pd.read_excel('../sentiment/sentiment_comparison.xlsx')

In [4]:
report1 = classification_report(df['GS'], bert['sentiment_pred'], output_dict=True)
report3 = classification_report(df['GS'], multilingual['MULTILINGUAL'], output_dict=True)

In [5]:
report1

{'negative': {'precision': 0.615678776290631,
  'recall': 0.7368421052631579,
  'f1-score': 0.6708333333333333,
  'support': 437},
 'neutral': {'precision': 0.3167808219178082,
  'recall': 0.2605633802816901,
  'f1-score': 0.28593508500772796,
  'support': 710},
 'positive': {'precision': 0.8848155156102177,
  'recall': 0.8932664756446992,
  'f1-score': 0.8890209125475286,
  'support': 4188},
 'accuracy': 0.7962511715089035,
 'macro avg': {'precision': 0.6057583712728856,
  'recall': 0.6302239870631824,
  'f1-score': 0.6152631102961966,
  'support': 5335},
 'weighted avg': {'precision': 0.7871740184022944,
  'recall': 0.7962511715089035,
  'f1-score': 0.790888033509129,
  'support': 5335}}

**Accuracy**
L'accuracy misura la percentuale di previsioni corrette sul totale delle previsioni. È semplice da interpretare, ma non è sempre il miglior indicatore delle prestazioni di un modello, specialmente in presenza di classi sbilanciate. Se le classi sono bilanciate, l'accuracy può essere una buona misura.

**F1-score**
L'F1-score è la media armonica della precisione e del recall. È particolarmente utile quando hai classi sbilanciate perché tiene conto sia dei falsi positivi che dei falsi negativi. Se il tuo dataset ha una distribuzione delle classi sbilanciata, l'F1-score è generalmente una scelta migliore rispetto all'accuracy.

In [6]:
# Estraggo l'affidabilità (ad esempio, F1-score) per ciascun modello e classe
reliabilities = {
    'bert': {label: {'f1-score': report1[label]['f1-score']} for label in ['negative', 'neutral', 'positive']},
    'multilingual': {label: {'f1-score': report3[label]['f1-score']} for label in ['negative', 'neutral', 'positive']}
}

In [7]:
reliabilities

{'bert': {'negative': {'f1-score': 0.6708333333333333},
  'neutral': {'f1-score': 0.28593508500772796},
  'positive': {'f1-score': 0.8890209125475286}},
 'multilingual': {'negative': {'f1-score': 0.686847599164927},
  'neutral': {'f1-score': 0.3286999182338512},
  'positive': {'f1-score': 0.9054069972906114}}}

In [8]:
# vado a sommare le probabilità di ciascuna stella in modo da ottenere un'unica probabilità per ciascun sentiment ( enon 5 diverse)
multilingual['negative'] = multilingual['1'] + multilingual['2']
multilingual['neutral'] = multilingual['3']
multilingual['positive'] = multilingual['4'] + multilingual['5']

In [9]:
multilingual

,Unnamed: 0,id,POI,rating,recensione,GS,1,2,3,4,5,MULTILINGUAL,negative,neutral,positive
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.004849,0.009446,0.063394,0.392164,0.530147,positive,0.014295,0.063394,0.922311
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.001022,0.001809,0.033151,0.438019,0.525999,positive,0.002831,0.033151,0.964018
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.004407,0.063242,0.514002,0.385526,0.032823,neutral,0.067649,0.514002,0.418349
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.010631,0.089289,0.340011,0.504914,0.055155,positive,0.099920,0.340011,0.560069
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.002611,0.004687,0.043096,0.394785,0.554821,positive,0.007298,0.043096,0.949606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.001865,0.002672,0.008780,0.132900,0.853783,positive,0.004537,0.008780,0.986683
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.127622,0.136910,0.161904,0.224093,0.349471,positive,0.264532,0.161904,0.573564
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.003115,0.006855,0.028588,0.260003,0.701439,positive,0.009970,0.028588,0.961442
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.014572,0.037150,0.088022,0.334005,0.526252,positive,0.051722,0.088022,0.860257


In [10]:
# Prevedo il sentiment delle recensioni nel test set utilizzando ciascun modello:

# Creo un dizionario di DataFrame
probs_df = {
    'bert': bert[['negative', 'neutral', 'positive']],
    'multilingual': multilingual[['negative', 'neutral', 'positive']]
}

# Converto i DataFrame in una struttura di liste di probabilità
def convert_to_probs_list(df):
    return df.values.tolist()

# Creo il dizionario di probabilità nel formato richiesto
probs = {
    'bert': convert_to_probs_list(probs_df['bert']),
    'multilingual': convert_to_probs_list(probs_df['multilingual'])
}

In [11]:
probs

{'bert': [[0.001214894349686801, 0.01089215371757746, 0.9878929257392883],
  [0.0001898571499623358, 0.004603196866810322, 0.9952070116996765],
  [0.0004285897011868656, 0.001523979124613106, 0.9980474710464478],
  [0.001172363292425871, 0.004624470602720976, 0.9942031502723694],
  [0.0005775829195044935, 0.05122018232941628, 0.9482021927833557],
  [0.0004616871592588723, 0.9811128377914429, 0.01842541806399822],
  [0.002698477124795318, 0.9433038830757141, 0.05399767681956291],
  [0.03105240315198898, 0.3384723961353302, 0.6304752230644226],
  [0.04519824683666229, 0.156735822558403, 0.7980659008026123],
  [0.0001330366358160973, 0.001792753464542329, 0.9980741739273071],
  [0.0001728587376419455, 0.001197158242575824, 0.9986300468444824],
  [0.0002516355598345399, 0.001612604595720768, 0.9981356859207153],
  [0.0001297822454944253, 0.006472351495176554, 0.9933978319168091],
  [0.0001857496099546552, 0.005086409859359264, 0.9947279095649719],
  [0.0003060690069105476, 0.01359954476356

In [12]:

# Applico il BMA per combinare le probabilità di ciascun modello
classes = {
    'bert': ['negative', 'neutral', 'positive'],
    'multilingual': ['negative', 'neutral', 'positive']
}
model_list = ['bert', 'multilingual']
source_tokens = df['recensione'].str.split().tolist()



In [13]:
len(source_tokens)

5335

In [14]:
source_tokens

[['Un',
  'palazzo',
  'appena',
  'restaurato,',
  'una',
  'ricca',
  'documentazione',
  'in',
  'ogni',
  'stanza',
  'e',
  "un'app",
  'per',
  'aiutare',
  'ancora',
  'di',
  'più.',
  'Non',
  'mancano',
  'gli',
  'affreschi,',
  'le',
  'documentazioni',
  'scientifiche',
  '...',
  'insomma',
  'è',
  'da',
  'non',
  'perdere'],
 ['Cinquecentesca',
  'dimora',
  'di',
  'una',
  'tra',
  'le',
  'famiglie',
  'illustri',
  'umbro',
  '-',
  'romane',
  '.',
  'Molto',
  'ben',
  'organizzati',
  'con',
  'ologrammi',
  'e',
  'proiezioni',
  'che',
  'guidano',
  "all'interno",
  'e',
  'in',
  'più',
  'noi',
  'abbiamo',
  'avuto',
  "l'occasione",
  'di',
  'avere',
  "un'oretta",
  'come',
  'guida',
  'il',
  'giovane',
  'Simone',
  '.',
  'Visita',
  'interessante',
  'e',
  'consigliata',
  '.'],
 ["L'imponente",
  'palazzo',
  'in',
  'stile',
  'rinascimentale',
  'è',
  'di',
  'per',
  'sé',
  'molto',
  'bello',
  'sia',
  'dal',
  'punto',
  'di',
  'vista',


In [15]:
predictions = bma_test(probs, reliabilities, model_list, classes, source_tokens)

100%|██████████| 5335/5335 [00:00<00:00, 186843.90it/s]


In [17]:
predictions

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',

In [19]:
df["pred_BMA"] = predictions

In [20]:
df

,id,POI,rating,recensione,GS,BERT,FEEL_IT,MULTILINGUAL,pred_BMA
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive,positive,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive,positive,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive,positive,neutral,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive,positive,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive,positive,positive,positive
...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive,positive,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive,positive,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive,positive,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive,positive,positive,positive


In [21]:
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)